This Google Colab notebook is made to help guide students through the concept of a Markov Decision Process (MDP) using the example of a grid maze. It also covers setup of a Gymnasium Environment and an implementation of Value Iteration.

In [ ]:
"""Install requisite packages"""
!pip install swig
!pip install gymnasium[box2d,classic-control]
!pip install numpy
!pip install matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 29.4 MB/s eta 0:00:00
  Created wheel for box2d-py: filename=box2d_py-2.3.5-cp310-cp310-linux_x86_64.whl size=2376098 sha256=24421b05a5196b4215b01b4a3b762729b6edc839f31ae5fa4b231dad0dee0785
  Stored in directory: /root/.cache/pip/wheels/db/8f/6a/eaaadf056fba10a98d986f6dce954e6201ba3126926fc5ad9e
Successfully built box2d-py


In [ ]:
"""Import packages"""
import io
import time

import ipywidgets
from IPython.display import display, clear_output
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

"""Utility functions"""
def image_to_bytes(image):
  img = Image.fromarray(image, 'RGB')

  # Convert the PIL image to bytes
  with io.BytesIO() as output:
    img.save(output, format="PNG")
    image_data = output.getvalue()
  return image_data

We want to represent a maze environment, where there is an agent that can move in the cardinal directions. There are walls which restrict movement. For a given maze of size WIDTHxHEIGHT, our MDP is defined as
- S: The state space is {0, 1, 2, ..., WIDTH*HEIGHT-1}, representing the column-wise flattened indices of the maze. For example, for a 5x5 maze, state 8 represents x=1, y=3.
- A: The action space is {0, 1, 2, 3}, representing the directions UP, DOWN, LEFT, and RIGHT respectively.
- R(S, A) -> ℝ The reward function takes a state and an action and gives a reward. For our simple problem, we give a constant penalty of -1 every action the agent takes. No reward is given at the goal.
- P(S, A, S) -> [0,1] The transition function shows the probability of starting at a given state, taking a certain action, and ending up in a different state. Our maze environment is deterministic, so all probabilities are either 0 or 1. For any non-neighbor states S1 and S2, P(S1, ⋅, S2)=0. But if state S3 is to the left of S4, then P(S3, LEFT, S4)=1.
- H: The horizon is the maximum amount of actions our agent is allowed to take.
- γ (gamma): Gamma is a value in [0,1] which acts as a discount factor to make states distant in the future less important. γ=1 means there is no discounting. γ=0.95 means that rewards lose 5% of their value each step into the future.

In [ ]:
class MazeMDP():

  MOVES: dict[int,tuple] = {
         0: np.array([-1, 0]), #UP
         1: np.array([1, 0]),  #DOWN
         2: np.array([0, -1]), #LEFT
         3: np.array([0, 1])   #RIGHT
         }

  def __init__(self, maze, H=100, gamma=.95):
    """Initialize the MDP"""
    self.maze = np.array(maze)

    self.obstacles = [tuple(loc) for loc in np.argwhere(self.maze == 1)]
    def get_state_index_from_maze(value):
      locations = [tuple(loc) for loc in np.argwhere(self.maze == value)]
      assert len(locations) == 1, f"There should be exactly one {value} in the maze"
      x, y = locations[0]
      return x * self.maze.shape[0] + y
    self.init_state = get_state_index_from_maze(2)
    self.goal_state = get_state_index_from_maze(3)

    """Create the MDP functions"""
    self.S = set(range(self.maze.shape[0] * self.maze.shape[1]))
    self.A = set(range(len(self.MOVES)))
    self.R = self.build_rewards()
    self.P = self.build_transitions()

    self.H = H
    self.gamma = gamma

  def build_rewards(self):
    """Build reward function"""
    # make every state and action give -1 reward, to penalize long paths
    rewards = np.full((len(self.S), len(self.A)), -1)
    # all actions in the goal state give zero reward
    rewards[self.goal_state, :] = 0

    return rewards

  def build_transitions(self):
    """Build transitoins function"""
    transitions = np.zeros((len(self.S), len(self.A), len(self.S)))

    # Go through all states and all actions
    for s in self.S:
      for a in self.A:
        # By default, the next state is the current state
        s_p = s

        # Convert state to maze index
        xy = np.array([s // self.maze.shape[1], s % self.maze.shape[1]])

        # Get new potential state
        xy_new = xy + self.MOVES[a]

        # Check that new state is valid
        is_in_bounds = 0 <= xy_new[0] < self.maze.shape[0] and 0 <= xy_new[1] < self.maze.shape[1]
        is_free_space = tuple(xy_new) not in self.obstacles

        # Compute the new s' if the transition is valid
        if is_in_bounds and is_free_space:
          s_p = xy_new[0] * self.maze.shape[1] + xy_new[1]

        # Assign transition probability to 1
        transitions[s,a,s_p] = 1

    return transitions

Now that we've clearly defined the MDP, we want to provide an interface for an agent to interact with the environment. This is quite a simple environment, but you can imagine that they get quite complex. Gymnasium is a package that provides a common Environment interface between many such environments, and other useful utilities. `gymnasium` is often abbreviated as `gym`, because it is a well-maintained fork of OpenAI's Gym. A `gym.Env` has the following interface:
- `observation_space`: The observation space defines what data types the states are represented by. For example, an integer or a vector of floats.
- `action_space`: The action space defines what data types the actions are represented by. It is common to have an integer represent different choices.
- `reset()` -> `observation, info`: The reset function marks the start of a new episode. It resets the environment back to its starting state. It returns the starting observation (and a dictionary of auxillary info).
- `step(action)` -> `observation, reward, termination, truncation, info`: The step function executes an action chosen by the agent. The resulting state is returned along with the reward for that action. Termination and truncation signal that the current episode is done (such as by reaching the goal or time running out).
- `render()`: Render is for visualization purposes and is optional.

Let's see `gymnasium` in action with a few of their built-in environments. You can refer to https://gymnasium.farama.org/ for more details.

In [ ]:
import gymnasium as gym

# Lots of environments to choose from. Here are some examples
# "CartPole-v1", "MountainCar-v0", "Acrobot-v1", "LunarLander-v2", "CarRacing-v2"
environment_id = "LunarLander-v2"
env = gym.make(environment_id, render_mode="rgb_array")

def agent_policy(observation):
  # You can choose actions with any method.
  # This just chooses a random action
  return env.action_space.sample()

print("Observation space:", env.observation_space)
print("Action space:", env.action_space)
# Jupyter notebook stuff to render the output
image_widget = ipywidgets.Image()
display(image_widget)

observation, info = env.reset()
for _ in range(500):
  action = agent_policy(observation)
  observation, reward, terminated, truncated, info = env.step(action)

  done = terminated or truncated
  if done:
    # restart the episode when the previous finishes
    observation, info = env.reset()

  # Render
  image = env.render()
  image_widget.value = image_to_bytes(image)

env.close()

Observation space: Box([-1.5       -1.5       -5.        -5.        -3.1415927 -5.
 -0.        -0.       ], [1.5       1.5       5.        5.        3.1415927 5.        1.
 1.       ], (8,), float32)
Action space: Discrete(4)


Image(value=b'')

Now that you are familiar with what Gymnasium does, let's define a gym environment for our previously defined MazeMDP class. In most cases, you would just build the MDP directly into the environment.

In [ ]:
"""Define the Maze Gym Environment"""
class MazeEnv(gym.Env):

  def __init__(self, mdp, render_mode = "ansi"):
    """Initialize the maze environment setting all variables and parsing the map"""
    self.mdp = mdp

    self.agent_curr_state = self.mdp.init_state
    self.steps_taken = 0

    # Set action space and observation space
    self.action_space = gym.spaces.Discrete(len(self.mdp.A))
    self.observation_space = gym.spaces.Discrete(len(self.mdp.S))

    # Rendering
    self.render_mode = render_mode

  def reset(self):
    """Reset the environment to its initial state"""
    self.agent_curr_state = self.mdp.init_state
    self.steps_taken = 0

    initial_obs = self.agent_curr_state
    initial_info = {}

    return initial_obs, initial_info

  def step(self, action):
    """Take an action in the environment"""

    assert 0 <= action < 4, "Action should be one of [0,1,2,3]"

    # Get reward according to state and action
    reward = self.mdp.R[self.agent_curr_state, action]

    # Sample next state weighted by transition probabilities
    self.agent_curr_state = np.random.choice(list(self.mdp.S), 1, p=self.mdp.P[self.agent_curr_state, action,:])[0]

    # Make outputs for gym environment
    next_obs = self.agent_curr_state

    self.steps_taken += 1
    has_reached_goal = self.agent_curr_state == self.mdp.goal_state
    has_reached_time_limit = self.steps_taken >= self.mdp.H
    terminated = has_reached_goal or has_reached_time_limit

    # truncated indicates that the episode was ended due to some external condition
    truncated = False
    # info can contain auxillary information for logging and debugging
    info = {}

    return next_obs, reward, terminated, truncated, info

  def render(self):
    """ Render the environment """

    if self.render_mode == "ansi":
      # ansi is text mode
      return render_text(self.agent_curr_state, self.mdp.maze)
    else:
      raise ValueError(f"Unsupported rendering mode {self.render_mode}")

def render_text(agent_state, maze):
  """Get a text form of the maze"""
  character_map = ["-", "X", "-", "G"]
  ncols, nrows = maze.shape

  string=f""
  for i in range(ncols):
    for j in range(nrows):
      s = (i * ncols + j)
      if s == agent_state:
        string += "A"
      else:
        string += character_map[maze[i, j]]
    string += "\n"

  return string


We can test out this environment with an agent that selects random actions.

In [ ]:
STEPS_PER_SECOND = 10
output = ipywidgets.Output()
display(output)

# Create the maze MDP. 0 is free space, 1 is walls,
# 2 is the starting location, 3 is the goal location
mdp = MazeMDP([[2,0,0,0,0],
               [1,1,1,1,0],
               [0,0,0,0,0],
               [0,1,0,1,1],
               [0,1,0,0,3]])

# Create the maze gym environment
maze_env = MazeEnv(mdp)

terminated = truncated = done = False
obs, info = maze_env.reset()
num_steps = 0
while not done:
  # sample a random action
  action = maze_env.action_space.sample()

  next_obs, reward, terminated, truncated, info = maze_env.step(action)
  done = terminated or truncated
  obs = next_obs

  # Render
  with output:
    clear_output(wait=True)
    print(maze_env.render())
  time.sleep(1/STEPS_PER_SECOND)
  num_steps += 1

print(f"Finished in {num_steps} steps")

Output()

Finished in 100 steps


Clearly the random agent had no idea what it was doing. Using the Value Iteration algorithm, we can find the optimal policy to choose the best actions. We will discuss more about how this algorithm works in future sessions.

In [ ]:
class ValueIterationAgent():
  """Agent class for solving tabular MDPs"""

  def __init__(self, env):
    """Initialize the agent. env should have a tabular mdp"""

    assert hasattr(env, 'mdp'), "Gymansium Environment does not have associated MDP"

    self.env = env
    self.mdp = env.mdp
    self.policy = None
    self.v = None


  def value_improvement(self, epsilon=.01):
    """Perform value improvement"""
    mdp = self.mdp

    # Randomly initialize v
    # Easiest initialization is all zeros
    v = np.zeros(len(mdp.S))

    # Loop until the horizon is reached
    for _ in range(mdp.H):

      old_v = v

      # Perform Bellman Backup
      v = np.max(np.sum( mdp.P * (mdp.R[:,:,np.newaxis] + mdp.gamma * v), axis=-1), axis=-1)
      # Set goal state value to 0
      v[mdp.goal_state] = 0

      # Exit if threshold is reached
      if np.max(np.abs(old_v-v)) < epsilon:
        break

    # Determine the best actions with bellman backups
    action_reward = np.sum(mdp.P * (mdp.R[:,:,np.newaxis] + mdp.gamma * v), axis=-1)

    # Compute the optimal policy
    best_actions = (action_reward - np.max(action_reward, axis=1)[:,np.newaxis]) == 0
    policy = best_actions * (1 / np.sum( best_actions, axis=1)[:,np.newaxis])

    self.policy = policy
    self.v = v

    return policy, v

  def get_action(self, state):
    """Get the action given by the policy"""

    # If policy is none, return a random action
    if self.policy is None:
      print("Warning: Value iteration policy has not been computed yet")
      return self.env.action_space.sample()

    # Get the action given by the policy
    action = np.random.choice(list(self.mdp.A), 1, p=self.policy[state,:])[0]

    return action

  def get_value(self, state):
    """Get the value of a given state"""

    return self.v[state]

Now we can try again with the ValueIterationAgent. You'll find that it solves it reliably and much faster.

In [ ]:
STEPS_PER_SECOND = 2
output = ipywidgets.Output()
display(output)

# Create the maze MDP. 0 is free space, 1 is walls,
# 2 is the starting location, 3 is the goal location
mdp = MazeMDP([[2,0,0,0,0],
               [1,1,1,1,0],
               [0,0,0,0,0],
               [0,1,0,1,1],
               [0,1,0,0,3]])

# Create the maze gym environment
maze_env = MazeEnv(mdp)

# Create and call a value iteration agent
agent = ValueIterationAgent(maze_env)
agent.value_improvement()

terminated = truncated = done = False
obs, info = maze_env.reset()
num_steps = 0
while not done:
  # sample a random action
  action = agent.get_action(obs)

  next_obs, reward, terminated, truncated, info = maze_env.step(action)
  done = terminated or truncated
  obs = next_obs

  if done:
    obs, info = maze_env.reset()
    done = False
    print("Finished in ", num_steps)
    num_steps = 0

  # Render
  with output:
    clear_output(wait=True)
    print(maze_env.render())
  time.sleep(1/STEPS_PER_SECOND)
  num_steps += 1

print(f"Finished in {num_steps} steps")

Output()

Finished in  11
Finished in  12
Finished in  12
Finished in  12
Finished in  12
Finished in  12


KeyboardInterrupt: 